In [ ]:
# %%
%load_ext autoreload
%autoreload 2

import torch
from src.dataset import load_and_preprocess_data
from src.models import EALSTM
from src.training import train_epoch, evaluate
from src.inference import predict_and_save_test_results
from src.config import MODELS_DIR, OUTPUT_DATA_DIR

# 1. Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 30
HIDDEN_DIM = 256
LEARNING_RATE = 1e-3

print(f"Using device: {DEVICE}")

# 2. Load Data
train_loader, val_loader, test_loader, station_ids = load_and_preprocess_data(sequence_length=365)

# 3. Initialize Model
# Dynamic Features: Precip, Tmax, Tmin (3)
# Static Features: Area, MeanElev, Glacier% (3)
model = EALSTM(input_dim_dyn=3, 
               input_dim_stat=3, 
               hidden_dim=HIDDEN_DIM).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(model)

# 4. Training Loop
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    # Train on 1990-2008
    train_loss = train_epoch(model, train_loader, optimizer, DEVICE)
    
    # Validate on 2009-2012
    val_loss = evaluate(model, val_loader, DEVICE)
    
    print(f"Epoch {epoch+1} | Train Loss (NSE*): {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    # Save Best Model based on Validation
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODELS_DIR / "best_model.pth")
        print("   --> Saved new best model")

# 4. Final Benchmark on Test Set (1980s + 2013-2022)
print("--- Generating Test Predictions ---")
model.load_state_dict(torch.load(MODELS_DIR / "best_model.pth"))
df_preds = predict_and_save_test_results(
    model,
    DEVICE,
    output_file=OUTPUT_DATA_DIR / "test_set_predictions.csv"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cpu
⏳ Loading datasets...
   Normalizing features & Calculating Basin variances...
   ✅ Saved normalization scalers to C:\Users\tbwil\Documents\School\MSc Geophysics\Thesis Project\models\scalers.json
   Train Days: 6940 | Val Days: 1461 | Test Days: 7305
✅ Data Ready.
   Train Samples: 916080
EALSTM(
  (input_gate_net): Linear(in_features=3, out_features=256, bias=True)
  (w_f): Linear(in_features=259, out_features=256, bias=True)
  (w_g): Linear(in_features=259, out_features=256, bias=True)
  (w_o): Linear(in_features=259, out_features=256, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (head): Linear(in_features=256, out_features=1, bias=True)
)


Training:   0%|          | 2/3579 [00:04<2:16:47,  2.29s/it]


KeyboardInterrupt: 